In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: XX
### Team Member Names:
### Team Strategy Chosen: __________(RISKY OR SAFE)

In [2]:
# read the ticker file
tickers = pd.read_csv("Tickers.csv",header=None)
# rename the column as "ticker"
tickers.columns=['ticker']
# create a list that contains all the tickers
ticker_lst = tickers['ticker'].tolist()

len(ticker_lst)

62

In [3]:
# define function call_tickers that takes a list of tickers and return a list of called tickers
def call_tickers(lst):
    # create an empty list to store the called tickers
    called_ticker_lst = []
    # write a for loop to iterate the ticker list
    for i in lst:
        # call each ticker and append it to the called ticker list
        called_ticker_lst.append(yf.Ticker(i))
    # return the called ticker list when finish
    return called_ticker_lst

# defein function filter_USD that takes a list of called tickers and return a list of only US listed tickers
def filter_USD(lst):
    # create a new list to store the us tickers
    usd_lst = []
    # write a for loop that iterates the called ticker list
    for i in lst:
        # use try and except to ignore the delisted tickers
        try:
            # if the currency of the stock is "USD"
            if i.info['currency'] == 'USD':
                # append the ticker to the usd list
                usd_lst.append(i)
        except:
            pass
    # return the usd list
    return usd_lst

# define function filter_daily_volumn that takes a called ticker list, start date, end date, 
# and returns a list of tickers with an average daily_volume of at least 10000 shares in that time period
def filter_daily_volume(lst,start,end):
    # write a for loop to iterate the called ticker list
    for i in lst:
        # extract the volume of the stock in the given time period and see if the avergae is less than 10000
        if i.history(start = start, end = end,interval='1d')['Volume'].mean() < 10000:
            # if yes, remove the ticker from the list
            lst.remove(i)
        # return the list with unqualified tickers removed
        return lst


    
def get_prices(lst,start,end):
    prices = pd.DataFrame()
    for i in lst:
        prices[str(i)[24:-1]] = i.history(start = start,end=end,interval = '1d')['Close']
    return prices

def get_std(lst):
    std = pd.DataFrame(columns = ('ticker','std'))
    for i in range(len(lst)):
        std.loc[i] = [str(lst[i])[24:-1], prices[str(lst[i])[24:-1]].std()]
    return std

def rank(df,column_name,method):
    df['rank'] = df[column_name].rank(method=method)
    df = df.sort_values(by='rank')
    return df

In [7]:
# call the tickers
called_tickers = call_tickers(ticker_lst)
# filter the usd tickers
usd_tickers = filter_USD(called_tickers)
# filter the tickers with required average daily volumn
good_tickers = filter_daily_volume(usd_tickers, '2021-07-02','2021-10-22')
# extract the closing prices for the good tickers
prices = get_prices(good_tickers,'2018-01-01','2021-10-25')
# calculate the std according to the closing prices
std = get_std(good_tickers)
# rank them in ascending order
std = rank(std,'std','min')

In [84]:
std

,ticker,std,rank
30,MON,0.060949,1.0
25,KMI,2.105146,2.0
48,T,2.516413,3.0
39,PFE,3.944101,4.0
29,MO,5.223670,5.0
26,KO,5.345633,6.0
20,CSCO,5.739902,7.0
8,BAC,5.788770,8.0
10,BK,6.082770,9.0
12,BMY,7.347553,10.0


,ticker,std,rank
30,MON,0.060949,1.0
25,KMI,2.105146,2.0
48,T,2.516413,3.0
39,PFE,3.944101,4.0
29,MO,5.223670,5.0
26,KO,5.345633,6.0
20,CSCO,5.739902,7.0
8,BAC,5.788770,8.0
10,BK,6.082770,9.0
12,BMY,7.347553,10.0


In [30]:
prices[str[0][24:-1]].std()

TypeError: 'type' object is not subscriptable

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.